<a href="https://colab.research.google.com/github/lavanyanandikonda/Business-Performance-Analytics/blob/main/Business%20Performance%20using%20pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql.functions import col,lit,when
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
spark=SparkSession.builder.appName('MySession').getOrCreate()

In [2]:
pip install pyspark

In [3]:
df=spark.read.csv("/content/Dataset.csv",header=True,inferSchema=True)

In [4]:
df.columns
df=df.na.drop()
df.columns

['rowid',
 'orderid',
 'orderdata',
 'shipdate',
 'shipmode',
 'customerid',
 'customername',
 'segment',
 'country',
 'city',
 'state',
 'postalcode',
 'region',
 'productid',
 'category',
 'subcategory',
 'productname',
 'sales',
 'quantity',
 'discount',
 'profit']

In [5]:
selection_df = df.select('region', 'subcategory', 'category', 'quantity', 'sales')
selection_df.printSchema()

root
 |-- region: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- sales: string (nullable = true)



In [6]:
selection_df = selection_df.withColumn('sales', col('sales').cast('double'))
selection_df = selection_df.withColumn('quantity', col('quantity').cast('double'))

In [7]:
selection_df.show()

+-------+-----------+---------------+--------+--------+
| region|subcategory|       category|quantity|   sales|
+-------+-----------+---------------+--------+--------+
|  South|  Bookcases|      Furniture|     2.0|  261.96|
|  South|     Chairs|      Furniture|     3.0|  731.94|
|   West|     Labels|Office Supplies|     2.0|   14.62|
|  South|     Tables|      Furniture|     5.0|957.5775|
|  South|    Storage|Office Supplies|     2.0|  22.368|
|   West|Furnishings|      Furniture|     7.0|   48.86|
|   West|        Art|Office Supplies|     4.0|    7.28|
|   West|     Phones|     Technology|     6.0| 907.152|
|   West|    Binders|Office Supplies|     3.0|  18.504|
|   West| Appliances|Office Supplies|     5.0|   114.9|
|   West|     Tables|      Furniture|     9.0|1706.184|
|   West|     Phones|     Technology|     4.0| 911.424|
|  South|      Paper|Office Supplies|     3.0|  15.552|
|   West|    Binders|Office Supplies|     3.0| 407.976|
|Central| Appliances|Office Supplies|     5.0|  

In [8]:
selection_df = selection_df.na.drop()

indexer = StringIndexer (inputCols=["region", "subcategory"], outputCols=["region_indexed", "sub-Category_indexed"])
df_i = indexer.fit(selection_df).transform(selection_df)

df_i.show()


+-------+-----------+---------------+--------+--------+--------------+--------------------+
| region|subcategory|       category|quantity|   sales|region_indexed|sub-Category_indexed|
+-------+-----------+---------------+--------+--------+--------------+--------------------+
|  South|  Bookcases|      Furniture|     2.0|  261.96|           3.0|                12.0|
|  South|     Chairs|      Furniture|     3.0|  731.94|           3.0|                 7.0|
|   West|     Labels|Office Supplies|     2.0|   14.62|           0.0|                 9.0|
|  South|     Tables|      Furniture|     5.0|957.5775|           3.0|                10.0|
|  South|    Storage|Office Supplies|     2.0|  22.368|           3.0|                 4.0|
|   West|Furnishings|      Furniture|     7.0|   48.86|           0.0|                 2.0|
|   West|        Art|Office Supplies|     4.0|    7.28|           0.0|                 5.0|
|   West|     Phones|     Technology|     6.0| 907.152|           0.0|          

In [9]:
fa= VectorAssembler (inputCols=['region_indexed', 'sub-Category_indexed', 'quantity', 'sales' ], outputCol="Independent Features")
output = fa.transform(df_i)

In [10]:
final_data = output.select("Independent Features", "sales")
final_data.na.drop()
final_data.show()

+--------------------+--------+
|Independent Features|   sales|
+--------------------+--------+
|[3.0,12.0,2.0,261...|  261.96|
|[3.0,7.0,3.0,731.94]|  731.94|
| [0.0,9.0,2.0,14.62]|   14.62|
|[3.0,10.0,5.0,957...|957.5775|
|[3.0,4.0,2.0,22.368]|  22.368|
| [0.0,2.0,7.0,48.86]|   48.86|
|  [0.0,5.0,4.0,7.28]|    7.28|
|[0.0,3.0,6.0,907....| 907.152|
|[0.0,0.0,3.0,18.504]|  18.504|
| [0.0,8.0,5.0,114.9]|   114.9|
|[0.0,10.0,9.0,170...|1706.184|
|[0.0,3.0,4.0,911....| 911.424|
|[3.0,1.0,3.0,15.552]|  15.552|
|[0.0,0.0,3.0,407....| 407.976|
| [2.0,8.0,5.0,68.81]|   68.81|
| [2.0,0.0,3.0,2.544]|   2.544|
|[2.0,4.0,6.0,665.88]|  665.88|
|  [0.0,4.0,2.0,55.5]|    55.5|
|  [0.0,5.0,2.0,8.56]|    8.56|
|[0.0,3.0,3.0,213.48]|  213.48|
+--------------------+--------+
only showing top 20 rows



In [11]:
train_data, test_data = final_data.randomSplit([0.80, 0.20])
regressor=LinearRegression (featuresCol='Independent Features', labelCol='sales')
regressor = regressor.fit(train_data)

In [12]:
predict_results = regressor.evaluate(test_data)

predict_results. predictions.show()

+--------------------+------+------------------+
|Independent Features| sales|        prediction|
+--------------------+------+------------------+
| [0.0,0.0,1.0,2.025]| 2.025| 2.025000000000205|
| [0.0,0.0,1.0,4.304]| 4.304| 4.304000000000205|
| [0.0,0.0,1.0,5.022]| 5.022| 5.022000000000205|
| [0.0,0.0,1.0,5.682]| 5.682|5.6820000000002056|
| [0.0,0.0,1.0,6.384]| 6.384| 6.384000000000206|
| [0.0,0.0,1.0,9.584]| 9.584| 9.584000000000206|
| [0.0,0.0,2.0,3.108]| 3.108|3.1080000000001693|
| [0.0,0.0,2.0,4.544]| 4.544| 4.544000000000169|
| [0.0,0.0,2.0,4.938]| 4.938| 4.938000000000169|
| [0.0,0.0,2.0,5.214]| 5.214|  5.21400000000017|
| [0.0,0.0,2.0,5.728]| 5.728| 5.728000000000169|
| [0.0,0.0,2.0,6.368]| 6.368| 6.368000000000171|
| [0.0,0.0,2.0,7.184]| 7.184| 7.184000000000171|
| [0.0,0.0,2.0,7.712]| 7.712|  7.71200000000017|
| [0.0,0.0,2.0,8.544]| 8.544|  8.54400000000017|
| [0.0,0.0,2.0,9.296]| 9.296| 9.296000000000168|
| [0.0,0.0,2.0,9.872]| 9.872| 9.872000000000169|
|[0.0,0.0,2.0,10.128

In [13]:
print("MAE is ", predict_results.meanAbsoluteError)
print("MSE is ", predict_results.meanSquaredError)
print("RMSE is ",predict_results.rootMeanSquaredError)
print("R2 is ",predict_results.r2)
print("Adj R2 is ", predict_results.r2adj)

MAE is  9.318813989752124e-14
MSE is  2.087577475992404e-26
RMSE is  1.4448451391039816e-13
R2 is  1.0
Adj R2 is  1.0
